In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

In [2]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
url = "https://residence-inn.marriott.com/locations/"
driver.get(url)
time.sleep(30)
# viewhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[3]/button')[0]
# viewhotels_button.click()
# northamhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[2]/pacsys-directory-listings/div/div[5]/a')[0]
# northamhotels_button.click()

In [3]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [4]:
stuff = soup.find_all('a', {'class', 'swell_vertical-xs'})
# stuff
hotel_list = []
for x in stuff:
    hotel = {}
    hotel['name'] = x.text.strip()
    hotel['url'] = x.attrs['href']
    hotel_list.append(hotel)
hotelsurl_df = pd.DataFrame(hotel_list)
hotelsurl_df

,name,url
0,Residence Inn Algiers Bab Ezzouar,https://www.marriott.com/ALGRI
1,Residence Inn San Jose Escazu,https://www.marriott.com/SJORI
2,Residence Inn Merida,https://www.marriott.com/MIDRI
3,Residence Inn Panama City,https://www.marriott.com/PTYRI
4,Residence Inn Ghent,https://www.marriott.com/GNERI
...,...,...
890,Residence Inn Milwaukee Downtown,https://www.marriott.com/MKERI
891,Residence Inn Milwaukee West,https://www.marriott.com/MKEWE
892,Residence Inn Casper,https://www.marriott.com/CPRRI
893,Residence Inn Rio de Janeiro Barra da Tijuca,https://www.marriott.com/RIORI


In [5]:
%%time

hotel_list = []
counter = 0
for index, row in hotelsurl_df.iterrows():
    name = row['name']
    url = row['url']
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        stuff = soup.find('div', {'class' : 'l-container l-header-container'})
#             hotel_list = []
        hotel = {}
        try:
            hotel['brand'] = 'Residence Inn'
        except:
            hotel['brand'] = 'Residence Inn'
        try:
            hotel['name'] = name
        except:
            hotel['name'] = name
        try:
            hotel['url'] = url
        except:
            hotel['url'] = url
        try:
            hotel['street'] = stuff.find_all('span')[2].text
        except:
            hotel['street'] = 'N/A'
        try:
            hotel['locality'] = stuff.find_all('span')[4].text
        except:
            hotel['locality'] = 'N/A'        
        try:
            hotel['state'] = stuff.find_all('span')[6].text
        except:
            hotel['state'] = 'N/A'            
        try:
            hotel['postalcode'] = stuff.find_all('span')[7].text
        except:
            hotel['postalcode'] = 'N/A'            
        try:
            hotel['coordinate1'] = stuff.find_all('span')[25].text
        except:
            hotel['coordinate1'] = 'N/A'            
        try:
            hotel['coordinate2'] = stuff.find_all('span')[26].text
        except:
            hotel['coordinate2'] = 'N/A'
        try:
            hotel['coordinate3'] = stuff.find_all('span')[27].text
        except:
            hotel['coordinate3'] = 'N/A'            
        try:
            hotel['coordinate4'] = stuff.find_all('span')[28].text
        except:
            hotel['coordinate4'] = 'N/A'            
        
        desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})
        try:
            hotel['description'] = str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')
        except:
            hotel['description'] = 'N/A'
        
        overallratings = stuff = soup.find('div', {'class' : 'l-overall-rating-container'})
        try:
            hotel['overall_rating'] = ratings.find_all('div')[0].text.strip()[:4]
        except:
            hotel['overall_rating'] = 'N/A'
        
        specratings = stuff = soup.find('div', {'class' : 'l-secondary-ratings-container l-clear l-margin-bottom-quarter l-m-margin-bottom-none'})
        try:
            hotel['cleanliness_rating'] = specratings.find_all('div')[2].text.strip()[:4]
        except:
            hotel['cleanliness_rating'] = 'N/A'
        try:
            hotel['service_rating'] = specratings.find_all('div')[7].text.strip()[:4]
        except:
            hotel['service_rating'] = 'N/A'
            
        hotel_list.append(hotel)
        print(hotel)
        print()
        print(datetime.now())
        print('='*80)
        print('='*80)
        print()
#         driver.close()
#         driver.switch_to.window(original_window)
#         time.sleep(1)

        counter += 1
        print(f'Collected {counter} hotels.')
    except:
        hotel = {}
        hotel['name'] = name
        hotel['url'] = url
        hotel_list.append(hotel)
        print(f"Number {x} didn't work at {datetime.now()}")
        continue
hotel_df = pd.DataFrame(hotel_list)
hotel_df        

{'brand': 'Residence Inn', 'name': 'Residence Inn Algiers Bab Ezzouar', 'url': 'https://www.marriott.com/ALGRI', 'street': 'Trust Complex Building, Nouveau Quartier des Affaires, Bab Ezzouar', 'locality': 'Algiers', 'state': '', 'postalcode': '16311', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Set a pace that suits you at Residence Inn Algiers Bab Ezzouar. Whether you're with us for a few days, a few weeks or a few months, your daily routine matters. You'll thrive at Residence Inn with the conveniences you need. Like spacious studio, one- and two-bedroom suites with separate living and sleeping areas. A fully equipped kitchen ready for your favourite cuisine. Complimentary grocery delivery service to fill your refrigerator and satisfy cravings. Free Internet for connecting to work and the outside world. Free hot breakfast to start mornings off right and evening social hours to end the day well. And, a fitness centre for str

{'brand': 'Residence Inn', 'name': 'Residence Inn Toulouse-Blagnac Airport', 'url': 'https://www.marriott.com/TLSTB', 'street': "11 Avenue d'Andromede, ZAC Andromede", 'locality': 'Blagnac', 'state': '', 'postalcode': '31700', 'coordinate1': '43.655592', 'coordinate2': '1.368811', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Located 3 kilometers from the main airport and just 15 minutes from Toulouse's main touristic attractions, Residence Inn Toulouse Blagnac Airport will be an ideal stop for both business and leisure travelers. 140 rooms and suites offer functionality and comfort to all travelers. You will enjoy an equipped kitchen in all our rooms, the ideal address for long stays. For a night, a week or a month, you will discover an intimate atmosphere with the comfort and quality expected of Marriott Hotels. Spacious suites with high-speed internet and kitchen, the complimentary breakfast and the fitness room will help you relax and feel right at home.Enjoy a meal

{'brand': 'Residence Inn', 'name': 'Residence Inn Aberdeen', 'url': 'https://www.marriott.com/ABZRI', 'street': 'Guestrow', 'locality': 'Aberdeen', 'state': '', 'postalcode': 'AB10 1AS', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/abzri-residence-inn-aberdeen/?directPageRequest=true', 'coordinate3': '57.148317', 'coordinate4': '-2.097497', 'description': '[ Placed in the city centre, Residence Inn Aberdeen is a contemporary haven for the on-the-go traveler. Discover well-appointed studios and suites, thoughtfully equipped for long-term business stays; within our accommodation, find full kitchens and a grocery delivery service, comfortable beds, deluxe linens, spacious workstations and complimentary Wi-Fi. Our family oriented hotel even allows you to bring your animal with you to our pet-friendly accommodation. Begin your morning in Aberdeen with a free Continental breakfast and stock up on everyday essentials at our 24/7 lobby market. Many attr

{'brand': 'Residence Inn', 'name': 'Residence Inn Manama Juffair', 'url': 'https://www.marriott.com/BAHRI', 'street': 'Building 894, Road 2414, Block 324, Al Fateh', 'locality': 'Manama', 'state': '', 'postalcode': '76151', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/bahri-residence-inn-manama-juffair/?directPageRequest=true', 'coordinate3': '26.219457', 'coordinate4': '50.604734', 'description': "[ Stay in a spacious hotel in Juffair, Bahrain that provides travelers with the luxury of a hotel and the comforts of home. Residence Inn Manama, Juffair is designed to meet your needs whether you're with us for a few days, a few weeks or months. Relax in a studio, one-, two- or three-bedroom suite with separate living and sleeping areas ideal for spending time with your family or entertaining guests. As a state-of-the-art Manama, Bahrain, hotel, we offer fully equipped kitchens to prepare your favorite meals and complimentary grocery and restaurant d

{'brand': 'Residence Inn', 'name': 'Residence Inn Calgary South', 'url': 'https://www.marriott.com/YYCCR', 'street': '3710 Market Street SE.', 'locality': 'Calgary', 'state': 'Alberta', 'postalcode': 'T3M 2P2', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/yyccr-residence-inn-calgary-south/?directPageRequest=true', 'coordinate2': '50.88029', 'coordinate3': '-113.95594', 'coordinate4': 'N/A', 'description': "[ Offering an ideal home base from which to explore Southern Alberta, Residence Inn Calgary South boasts a convenient location near Chinook Centre, Spruce Meadows, Calgary Tower and Heritage Park Historical Village. Our modern hotel features spacious suites with homelike amenities including free high-speed Wi-Fi, ergonomic workstations, flat-panel TVs and pullout sofa beds. Fully equipped kitchens allow you to prepare delicious meals just like you were at home. If you don't feel like cooking in the morning, you can our hearty and healthy options at our complimentary br

...

{'brand': 'Residence Inn', 'name': 'Residence Inn Seattle Downtown/Convention Center', 'url': 'https://www.marriott.com/SEAAV', 'street': '1815 Terry Avenue', 'locality': 'Seattle', 'state': 'Washington', 'postalcode': '98101', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/seaav-residence-inn-seattle-downtown-convention-center/?directPageRequest=true', 'coordinate2': '47.615494', 'coordinate3': '-122.332953', 'coordinate4': 'N/A', 'description': '[ Experience Seattle’s unique, creative energy from our brand new hotel. Built from the ground up and designed with style, comfort and added amenities, this hotel merges functionality with an urban lifestyle. Maximize your extended stay in our apartment-style suites, with the convenience of a fully equipped kitchen. After an active day of work or adventure in the Pacific Northwest mountains, kick back with friends while enjoying a cold craft beer and local cuisine at our lobby bar.Every morning, fuel up for the day ahead with our

{'brand': 'Residence Inn', 'name': 'Residence Inn Portland Vancouver', 'url': 'https://www.marriott.com/PDXVR', 'street': '411 SE 123rd Avenue', 'locality': 'Vancouver', 'state': 'Washington', 'postalcode': '98683', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/pdxvr-residence-inn-portland-vancouver/?directPageRequest=true', 'coordinate2': '45.618553', 'coordinate3': '-122.545181', 'coordinate4': 'N/A', 'description': "[ Whether you're here for business or leisure, transform your getaway into an extended vacation at Residence Inn Portland Vancouver. Our location provides convenient access to many of the area's most beloved attractions, including the Columbia River, Multnomah Falls and Crowne Point. With our close proximity to Portland International Airport, you'll find that your stay begins virtually as soon as your plane touches down the ground. Take a moment to yourself in the modern decor of our spacious suites, featuring separate living areas, fully equipped kitchens 

{'brand': 'Residence Inn', 'name': 'Residence Inn Milwaukee Brookfield', 'url': 'https://www.marriott.com/MKERB', 'street': '765 Pinehurst Court', 'locality': 'Brookfield', 'state': 'Wisconsin', 'postalcode': '53005', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ The Residence Inn Milwaukee Brookfield is nestled back in a quiet resort-style setting with breathtaking views of the executive golf course that surrounds our property.  The hotel is designed as an experience where you can truly live your life on the road. Located off I-94 &amp; Moorland Road minutes away from downtown Milwaukee - our Waukesha County extended stay hotel sits in the region’s 2nd largest county and one of the primary economic engines in the state.   The strip of Bluemound Road in Brookfield is bustling with a multitude of shopping, dining and entertainment - anchored by the ever expanding Brookfield Square on the east, The Corners on the west and The Co

{'brand': 'Residence Inn', 'name': 'Residence Inn Milwaukee Downtown', 'url': 'https://www.marriott.com/MKERI', 'street': '648 North Plankinton Avenue', 'locality': 'Milwaukee', 'state': 'Wisconsin', 'postalcode': '53203', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/mkeri-residence-inn-milwaukee-downtown/?directPageRequest=true', 'coordinate2': '43.038046', 'coordinate3': '-87.91116', 'coordinate4': 'N/A', 'description': '[ As your top choice for your extended stay needs, the Residence Inn by Marriott® Milwaukee Downtown boasts spectacular downtown and river views from our suites featuring 20-foot ceilings and full-length windows highlighting the dramatic downtown Milwaukee architecture.  The function rooms compliment the hotel’s host of current amenities for our guests needing to host meetings. The multi-million dollar renovation has created a modern and comfortable place to stay. Experience our spacious river walk view studio, one and two bedroom suites offering full 

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Residence Inn,Residence Inn Algiers Bab Ezzouar,https://www.marriott.com/ALGRI,"Trust Complex Building, Nouveau Quartier des A...",Algiers,,16311,N/A,N/A,N/A,N/A,[ Set a pace that suits you at Residence Inn A...,N/A,N/A,N/A
1,Residence Inn,Residence Inn San Jose Escazu,https://www.marriott.com/SJORI,Ave. Escazu Complex Prospero Fernandez Hwy,San Jose,,10203,Reviews,https://www.marriott.com/hotels/maps/travel/sj...,9.938308,-84.142597,[ Find the comforts of home at Residence Inn S...,N/A,4.8,4.8
2,Residence Inn,Residence Inn Merida,https://www.marriott.com/MIDRI,"Calle 3-A No 272 X 20 A, Colonia Xcumpich",Merida,,97204,N/A,N/A,N/A,N/A,[ Residence Inn by Marriott Merida is an exten...,N/A,N/A,N/A
3,Residence Inn,Residence Inn Panama City,https://www.marriott.com/PTYRI,"Calle Ramon H Jurado, Calle Tomas Gabriel Duqu...",Panama City,,,N/A,N/A,N/A,N/A,[ Residence Inn Panama City is an extended-sta...,N/A,4.8,4.8
4,Residence Inn,Residence Inn Ghent,https://www.marriott.com/GNERI,Akkerhage 2A,Ghent,,9000,N/A,N/A,N/A,N/A,[ Set a pace that suits you at Residence Inn b...,N/A,4.2,4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,Residence Inn,Residence Inn Milwaukee Downtown,https://www.marriott.com/MKERI,648 North Plankinton Avenue,Milwaukee,Wisconsin,53203,https://www.marriott.com/hotels/maps/travel/mk...,43.038046,-87.91116,N/A,[ As your top choice for your extended stay ne...,N/A,4.5,4.7
891,Residence Inn,Residence Inn Milwaukee West,https://www.marriott.com/MKEWE,1300 Discovery Parkway,Wauwatosa,Wisconsin,53226,https://www.marriott.com/hotels/maps/travel/mk...,43.048197,-88.03109,N/A,[ Set a pace that suits you at Residence Inn b...,N/A,4.5,4.5
892,Residence Inn,Residence Inn Casper,https://www.marriott.com/CPRRI,4930 East 2nd Street,Casper,Wyoming,82609,42.847751,-106.260228,N/A,N/A,"[ Just a short drive from Halliburton, Sinclai...",N/A,4.9,4.9
893,Residence Inn,Residence Inn Rio de Janeiro Barra da Tijuca,https://www.marriott.com/RIORI,Av Embaixador Abelardo Bueno 5001,Rio de Janeiro,,22775-039,-22.974083,-43.400452,N/A,N/A,[ Treat yourself to an elevated travel experie...,N/A,4.3,4.4


In [6]:
hotel_df = hotel_df.drop_duplicates(subset='name')

In [7]:
hotel_df

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Residence Inn,Residence Inn Algiers Bab Ezzouar,https://www.marriott.com/ALGRI,"Trust Complex Building, Nouveau Quartier des A...",Algiers,,16311,N/A,N/A,N/A,N/A,[ Set a pace that suits you at Residence Inn A...,N/A,N/A,N/A
1,Residence Inn,Residence Inn San Jose Escazu,https://www.marriott.com/SJORI,Ave. Escazu Complex Prospero Fernandez Hwy,San Jose,,10203,Reviews,https://www.marriott.com/hotels/maps/travel/sj...,9.938308,-84.142597,[ Find the comforts of home at Residence Inn S...,N/A,4.8,4.8
2,Residence Inn,Residence Inn Merida,https://www.marriott.com/MIDRI,"Calle 3-A No 272 X 20 A, Colonia Xcumpich",Merida,,97204,N/A,N/A,N/A,N/A,[ Residence Inn by Marriott Merida is an exten...,N/A,N/A,N/A
3,Residence Inn,Residence Inn Panama City,https://www.marriott.com/PTYRI,"Calle Ramon H Jurado, Calle Tomas Gabriel Duqu...",Panama City,,,N/A,N/A,N/A,N/A,[ Residence Inn Panama City is an extended-sta...,N/A,4.8,4.8
4,Residence Inn,Residence Inn Ghent,https://www.marriott.com/GNERI,Akkerhage 2A,Ghent,,9000,N/A,N/A,N/A,N/A,[ Set a pace that suits you at Residence Inn b...,N/A,4.2,4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,Residence Inn,Residence Inn Milwaukee Downtown,https://www.marriott.com/MKERI,648 North Plankinton Avenue,Milwaukee,Wisconsin,53203,https://www.marriott.com/hotels/maps/travel/mk...,43.038046,-87.91116,N/A,[ As your top choice for your extended stay ne...,N/A,4.5,4.7
891,Residence Inn,Residence Inn Milwaukee West,https://www.marriott.com/MKEWE,1300 Discovery Parkway,Wauwatosa,Wisconsin,53226,https://www.marriott.com/hotels/maps/travel/mk...,43.048197,-88.03109,N/A,[ Set a pace that suits you at Residence Inn b...,N/A,4.5,4.5
892,Residence Inn,Residence Inn Casper,https://www.marriott.com/CPRRI,4930 East 2nd Street,Casper,Wyoming,82609,42.847751,-106.260228,N/A,N/A,"[ Just a short drive from Halliburton, Sinclai...",N/A,4.9,4.9
893,Residence Inn,Residence Inn Rio de Janeiro Barra da Tijuca,https://www.marriott.com/RIORI,Av Embaixador Abelardo Bueno 5001,Rio de Janeiro,,22775-039,-22.974083,-43.400452,N/A,N/A,[ Treat yourself to an elevated travel experie...,N/A,4.3,4.4


In [8]:
hotel_df.to_csv('residenceinn_description.csv')